<a href="https://colab.research.google.com/github/skadina/VTI-Data/blob/main/notebooks/NLPPipeSimple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP Pipe
### Vienkāršots piemērs NLP Pipe rīka pielietošanai
Šajā lapā

In [ ]:
# Augšuplādējam google colab vidē teksta failu
!wget https://raw.githubusercontent.com/LUMII-AILab/NLP_Course/main/notebooks/resources/velnini.txt
# filename = "velnini.txt"

In [ ]:
# Importējam kodam nepieciešamās bibliotēkas/palīgrīkus
import requests
import urllib
import json

In [ ]:
# Nolasām teksta failā esošo tekstu
file = open(filename, 'r', encoding="utf-8")
text = file.read()

# Lai pārbaudītu, ka teksts ir pareizi nolasīts, izdrukājam mainīgā "text" saturu
print(text)

### NLP-Pipe izsaukšana
Šī ir galvenā funkcija, kura nosūta tekstu NLP-Pipe rīkam un nolasa saņemto atbildi.

In [ ]:
# Nodefinējam funkciju, kas izsauc LV-PIPE REST API
def process_lvpipe(text, steps=None, api_url="https://nlp.ailab.lv/api/nlp"):
#    steps = steps or ['tokenizer', 'morpho', 'parser'] # +ner
    steps = steps or ['tokenizer', 'morpho'] # 'parser',  +ner

    response = requests.post(api_url, json={'data': {'text': text}, 'steps': steps})
    return response.json()['data']

Iepriekš izveidotajai funkcijai ir iespējams padot tekstu, kuru nepieciešams analizēt.

In [ ]:
# Iepriekš no faila nolasīto tekstu analizējam ar NLP-Pipe rīku.
doc = process_lvpipe(text)

# Rakstām visu NLP-Pipe rīka atgriezto informāciju failā.
with open("lvpipe_output.json", 'w') as f:
    json.dump(doc, f, indent=4, ensure_ascii=False)

# Apskatām kādu informāciju ir atgriezis NLP-Pipe rīks – izprintēsim šī dokumeneta saturu.
with open("lvpipe_output.json", 'r') as f:
    print(f.read())

# Rezultātu var arī apskatīt failā "lvpipe_output.json", kam vajadzētu atrasties pie google colab failiem.
# Ja neparādās, tad var nospiest pogu "refresh", lai atjauninātu mapes saturu.

Lai gan iegūtajā rezultātā jau ir visa analizētā informācija, JSON formāts nav labi cilvēkam pārskatāms.<br>
Tāpēc pievienoju arī papildus kodu, kas pārveido šo informāciju uz CoNLL formātu. <br>
Šis kods ir apslēpts, tādēļ ka ir diezgan garš, bet principā viss ko tas dara ir pārkārto informāciju no vienas pieraksta formas uz otru.

In [ ]:
# @title Pārveidošana no JSON uz CoNLL {"display-mode":"form"}
# Most of the work in NLP is to convert data from one format to another ;)

# This code block allows for customisation and format conversion,
# since other NLP pipelines might use different data formats and naming conventions.

# ID     = "index"
FORM   = "form"
LEMMA  = "lemma"
# UPOS   = "upos"
XPOS   = "tag"
# FEATS  = "ufeats"
# HEAD   = "parent"
# DEPREL = "deprel"
# DEPS   = "deps"
# MISC   = "misc"

# FORMAT = [ID, FORM, LEMMA, UPOS, XPOS, FEATS, HEAD, DEPREL, DEPS, MISC]
FORMAT = [FORM, LEMMA, UPOS, XPOS]


# For each token, creates a TAB-separated list of its UD features
def token_features(token):
    line = ""
    cols = FORMAT
    for x in cols:
        if token.get(x): line += str(token[x]) + '\t'
#        else: line += "_\t"
    return line.lstrip('\t') + '\n'


# De-tokenizes a tokenized sentence
def detokenize_sentence(sentence):
    text = ""
    for token in sentence["tokens"]:
        text += token[FORM] + ' ' # Naive glueing
    return text.lstrip(' ')


# Reads a JSON file produced by LV-PIPE and converts it to the CoNLL-U format
def json_to_conll(json_file, conll_file):
    with open(json_file, encoding="utf-8") as j_file:
        data = json.load(j_file)

    with open(conll_file, 'w', encoding="utf-8") as c_file:
        for sentence in data["sentences"]:
            c_file.write("# text = {}\n".format(detokenize_sentence(sentence)))

            for token in sentence["tokens"]:
                c_file.write(token_features(token))

            c_file.write('\n')

In [ ]:
# @title CoNLL faila izprintēšana {"display-mode":"form"}
# Pretty-prints a CoNLL-U file
def print_conllu(conll_file):
    with open(conll_file, 'r') as f:
        lines = f.readlines()

    structured_lines = []

    for line in lines:
        if line.startswith("#"):
            # Comment lines
            structured_lines.append(line.strip())
        elif line.strip():
            # Non-empty non-comment lines
            structured_lines.append(line.strip().split("\t"))
        else:
            # Empty lines - sentence breaks
            structured_lines.append('')

    # Find the maximum width of each column
    data_rows = [row for row in structured_lines if isinstance(row, list)]
    max_widths = [max(len(row[i]) for row in data_rows) for i in range(len(data_rows[0]))]

    for line in structured_lines:
        if isinstance(line, list):
            # A data row
            print(' '.join(word.ljust(max_widths[i]) for i, word in enumerate(line)))
        else:
            # A comment or empty line
            print(line)
            if line == "": print()

In [ ]:
# Izsaucam pārveidošanu no JSON formāta uz CoNLL formātu
json_to_conll("lvpipe_output.json", "lvpipe_output.conll")

# Lai pārbaudītu, ka teksts ir pareizi analizēts, izprintējam CoNLL faila saturu
print_conllu("lvpipe_output.conll")

# Rezultātu var arī apskatīt failā "lvpipe_output.conll", kam vajadzētu atrasties pie google colab failiem.
# Ja neparādās, tad var nospiest pogu "refresh", lai atjauninātu mapes saturu.